# Multiforest optimization notebook

Above the code cells, there will be instructions how the users should modify the codes in the cells. If there are no instructions, then by default no changes should be needed for the cell.

## Read the data

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/py_class")

import multiFunctionalOptimizationSwe as MFO

In [2]:
from importlib import reload
reload(MFO)

<module 'multiFunctionalOptimizationSwe' from '/home/ubuntu/workspace/mf_optimization_demo/py_class/multiFunctionalOptimizationSwe.py'>

You can choose solver by defining "solver=XXX" in the argument. Possible options are "CPLEX", "CLP" and "GLOP"

In [3]:
mfo = MFO.MultiFunctionalOptimization(solver = "CPLEX")

'Using CPLEX'

Define CC scenario data

In [4]:
RCP = "RCP0"
objectives_globiom = 'globiom_RCP0_V2'

In [5]:
#RCP = "RCP45"
#objectives_globiom = 'globiom_RCP45_V2'

1.5 degrees = RCP0 (no CC), NDC = RCP 4.5

In [6]:
scenario = "BAU"

In [7]:
import wget
import os
import numpy as np
import pandas as pd

NoCC:

In [8]:
%%time
mfo.readData("Sweden_5%_RCP0.zip",
             standsEnu = "Description", regimesEnu = ["ControlCategoryName", "AlternativeNo"], timeEnu = "period",
             sampleRatio = 0.5, #If no sample ratio given, the ratio is assumed to be 1
             areaCol = "RepresentedArea"
            )

'sample size 747/1494(50%)'

CPU times: user 2.97 s, sys: 964 ms, total: 3.93 s
Wall time: 3.93 s


RCP45:

In [9]:
#%%time
#mfo.readData("Sweden_5%_RCP45.csv",
#             standsEnu = "Description", regimesEnu = ["ControlCategoryName", "AlternativeNo"], timeEnu = "period",
#             sampleRatio = 1,
#             areaCol = "RepresentedArea"
#            )

Remove forestry regimes connected to intensification:

In [10]:
indexNames = mfo.data[(mfo.data['ControlCategoryName'] == 'Int_Prod') | (mfo.data['ControlCategoryName'] == 'Int_HybridExotic') | (mfo.data['ControlCategoryName'] == 'Int_Contorta')].index
mfo.data.drop(indexNames, inplace = True)

Create a column for Pulpfuel i.e. pulp plus firewood

In [11]:
mfo.data['PulpFuel'] = mfo.data.SumPulpVolumeTotal.values + mfo.data.SumHarvestFuelwoodTotal.values

Create column for simulated harvest for globiom optimisation

In [12]:
mfo.data['SimulatedSAWlog'] = mfo.data.SumTimberVolumeTotal.values/5

In [13]:
mfo.data['SimulatedResidue'] = mfo.data.SumHarvestResiduesTotal.values/5

In [14]:
mfo.data['SimulatedPulPFuel'] = mfo.data.PulpFuel.values/5

Create a new column for old decidious forest: older than 80 years and more than 30% deciduous

In [15]:
mfo.data['DeciduousRatio'] = mfo.data.VolumeDecidous.values/mfo.data.StandingVolume.values

<ipython-input-15-9487c82f8522>:1: RuntimeWarning: invalid value encountered in true_divide
  mfo.data['DeciduousRatio'] = mfo.data.VolumeDecidous.values/mfo.data.StandingVolume.values


In [16]:
mfo.data["old_deciduous_rich_forest_area"] = (mfo.data['DeciduousRatio'].values>0.3)*(mfo.data["Age"].values>80)*mfo.data["RepresentedArea"].values

Create bolean indicator for set asides to calculate share of forest set aside from management

In [17]:
mfo.data['SetAside'] = np.where(mfo.data.ControlCategoryName == 'SetAside (Unmanaged)',True,False)

Create periodic increment on managed land to be used in the constraints section further down

In [18]:
mfo.data['managed'] = np.where(mfo.data.ControlCategoryName == 'SetAside (Unmanaged)',False,True)

In [19]:
mfo.data['PeriodicIncrementNetTotal_managed'] = mfo.data.AnnualIncrementNetTotal.values*5*mfo.data.managed.values

Create an upper and lower limit for harvest volumes: 90-110% of the periodic increment

In [20]:
mfo.data['110%_of_periodic_increment_managed'] = mfo.data.AnnualIncrementNetTotal.values*5*mfo.data.managed.values*1.1
mfo.data['90%_of_periodic_increment_managed'] = mfo.data.AnnualIncrementNetTotal.values*5*mfo.data.managed.values*0.9

Manual calculation of Total values:

In [21]:
mfo.data["Total_VolumeDeciduous"] = mfo.data["VolumeDecidous"] * mfo.data["RepresentedArea"]
mfo.data["Total_DeadWoodVolume"] = mfo.data["DeadWoodVolume"] * mfo.data["RepresentedArea"]
mfo.data["Total_RecreationIndex"] = mfo.data["RecreationIndex"] * mfo.data["RepresentedArea"]
mfo.data["Total_TotalCarbon"] = mfo.data["TotalCarbon"] * mfo.data["RepresentedArea"]
mfo.data["Total_SimulatedSAWlog"] = mfo.data["SimulatedSAWlog"] * mfo.data["RepresentedArea"]
mfo.data["Total_SimulatedPulPFuel"] = mfo.data["SimulatedPulPFuel"] * mfo.data["RepresentedArea"]
mfo.data["Total_SimulatedResidue"] = mfo.data["SimulatedResidue"] * mfo.data["RepresentedArea"]

## Finalise data:

In [22]:
mfo.finalizeData(initialTime=0, initialRegime="ControlCategoryNameInitial state_AlternativeNo1")

## Start defining the optimization problem

#### Define objectives

Objective format: Unique_key : (Long human readable name,column name in data, max/min objective, year wise aggregation, stand wise aggregation [, target year])

Options for "objective": "max"imise or "min"imise it
year wise aggregation: "min" (minimum value), "average", "firstYear", "sum", "targetYearWithSlope","targetYear"
stand wise aggregation: "sum", "areaWeightedAverage", "areaWeightedSum"
targe yeart: any year except the first one

Objective dictionary structure: "Unique short name":("Long human readable name","column name in the data")

In [23]:
Wood = {
"NetPresentValue":("Total sum net present value of cut forest","NPV","max","firstYear","areaWeightedSum"),
"HarvestEvenFlow":("Average harvest","SumVolumeCutTotal","max","min","areaWeightedAverage")
}

In [24]:
Recreation = { 
"RecreationIndex":("No decrease in recreation index","Relative_Total_RecreationIndex","max","min","sum")
}

In [25]:
Climate = { 
"TotalCarbon":("No decrease carbon stocks","Relative_Total_TotalCarbon","max","min","sum")
}

In [26]:
Biodiversity = {
"DeadWoodVolume":("No decrease in deadwood volume","Relative_Total_DeadWoodVolume","max","min","sum"),  
"OldDeciduous":("No decrease in old deciduous area","Relative_old_deciduous_rich_forest_area","max","min","sum"),
"SetAside":("Share of set aside forest","SetAside","max","firstYear","areaWeightedAverage")
}

In [27]:
Resilience = {
"DeciduousVolume":("No decrease in volume deciduous trees","Relative_Total_VolumeDeciduous","max","min","sum")
}

In [28]:
objectives = {
    **Wood,
    **Recreation,
    **Climate,
    **Biodiversity,
    **Resilience
}

In [29]:
mfo.defineObjectives(objectives)

'Defining objectives'

'Aggregating stand wise'

100%|██████████| 8/8 [00:35<00:00,  4.44s/it]


'Aggregating year wise'

100%|██████████| 8/8 [00:00<00:00, 3477.14it/s]


'Objectives added'

GLOBIOM demands VERSION 2 - option for assortment transfer Attention: Has to be run after defining the objectives!

In [30]:
# 1.5 degree scenario, matches with RCP 0 (no CC)
if objectives_globiom == 'globiom_RCP0_V2':
    
    demands = pd.read_csv("G1p5_5%.csv") 

    print("used RCP0")
      
# NDC scenario, matches with RCP 4.5
elif objectives_globiom == 'globiom_RCP45_V2':
    
    demands = pd.read_csv("G4p5_5%.csv") 

    print("used RCP4.5")

used RCP0


In [31]:
if objectives_globiom == 'globiom_RCP0_V2' or objectives_globiom =='globiom_RCP45_V2' :
    
    sawlog = demands["GSawlog_uB"]
    sawlog = sawlog.to_list()
    
    pulpfuel = demands["GPulpFuel_uB"]
    pulpfuel = pulpfuel.to_list()

    residues = demands["GResidues"]
    residues = residues.to_list()
    
    mfo.addGlobiomTargets(
        {
        "log": sawlog,
        "pulp": pulpfuel,
        "residues": residues
        },
        {
            #Log is converted primarily into log, and sencondary into pulp; no transferrate, both are volumes under bark
            "Total_SimulatedSAWlog":
                {"log":[1,"primary"],"pulp":[1,"secondary"], "residues":[1.136,"secondary"]},
                                          
            # Pulp is converted primarily into pulp, and sencondary into residues; 
            # with transferrate 1.136, because pulp volume is under bark, residues are over bark (barkfactor = 1.136)  
            "Total_SimulatedPulPFuel":
                {"pulp":[1,"primary"], "residues":[1.136,"secondary"]}, 
            
            "Total_SimulatedResidue":
                {"residues":[1,"primary"]} #Biomass only to residues
        },
        
        # ----------------
        # by default exactMatching is FALSE
        # ----------------
        # Functionality was implemented for Norway and their nationl policy scenarios
        # NOT required for Cross-scale analysis V2
        exactMatching=False   
        
    )
    
    print("objective function with assortment transfer loaded")

objective function with assortment transfer loaded


#### Define constraints

In [32]:
constraints = {
"HarvestGrowth_upper":["less than", "Each period maximally 110% of the growth is harvested","SumVolumeCutTotal","areaWeightedSum","110%_of_periodic_increment_managed","areaWeightedSum"],
"HarvestGrowth_lower":["less than", "Each period minimally 90% of the growth is harvested","90%_of_periodic_increment_managed","areaWeightedSum","SumVolumeCutTotal","areaWeightedSum",]
}

In [33]:
mfo.defineConstraints(constraints)

## Calculate objective ranges

In [34]:
%%time
mfo.calculateObjectiveRanges()

'Calculating objective ranges'

  0%|          | 0/14 [00:00<?, ?it/s]

'Optimizing for Total sum net present value of cut forest'

'Found an optimal solution in 1 seconds'

'Objective values are:'

...

'Usage of Total_SimulatedSAWlog as residues in Globiom demands'

0.0

'Usage of Total_SimulatedPulPFuel as residues in Globiom demands'

0.0

100%|██████████| 14/14 [00:11<00:00,  1.18it/s]
CPU times: user 1min 11s, sys: 8.87 s, total: 1min 20s
Wall time: 11.9 s


In [35]:
mfo.objectiveRanges

{'NetPresentValue': (-25772374.46, 48848206539.48854),
 'HarvestEvenFlow': (0.0, 23.95161803908565),
 'RecreationIndex': (0.8571718773480209, 1.0327144367607461),
 'TotalCarbon': (0.9376711580951151, 1.0340794878911317),
 'DeadWoodVolume': (0.9018113436600184, 0.9927069053872193),
 'OldDeciduous': (0.19035109310690707, 1.074017583563738),
 'SetAside': (0.04039318546719784, 1.0000000000000018),
 'DeciduousVolume': (0.8700196549835925, 1.1046190917901684),
 'GlobiomTargetForlog': (0.0, 0.8959937247719038),
 'GlobiomTargetForpulp': (0.0, 1.954902167166046),
 'GlobiomTargetForresidues': (0.0, 7.722873470706),
 'GlobiomSecondaryUsageofTotal_SimulatedSAWlogaspulp': (0.0,
  41600832.595926836),
 'GlobiomSecondaryUsageofTotal_SimulatedSAWlogasresidues': (0.0,
  44769714.60601586),
 'GlobiomSecondaryUsageofTotal_SimulatedPulPFuelasresidues': (0.0,
  44434597.7856446)}

## Export the objetive ranges

Can save re-calculation times if big data sets are optimised

In [36]:
import json
mfo.objectiveRanges

with open("objectiveRanges_V2_"+RCP+"_"+scenario+".json", "w") as json_file:
    json.dump(mfo.objectiveRanges, json_file)

Save the objectives ranges also as CSV

In [37]:
import pandas
df = pandas.read_json("objectiveRanges_V2_"+RCP+"_"+scenario+".json")
df.to_csv("objectiveRanges_V2_"+RCP+"_"+scenario+".csv")

## Show the GUI

In [38]:
mfo.showGUI(debug=True)

interactive(children=(FloatSlider(value=-25772374.46, description='Total sum net present value of cut forest',…

interactive(children=(FloatSlider(value=24411217082.51427, description='Total sum net present value of cut for…

interactive(children=(Checkbox(value=False, description='Each period maximally 110% of the growth is harvested…

Button(description='Print solution', style=ButtonStyle())

## Export solution data as csv

In [39]:
import os

b = []
c = []
for key in mfo.regimesDecision.keys():
    if mfo.regimesDecision[key].solution_value() > 0:
        b = b+ [(key[0],x, key[1]) for x in range(0,21)]
        c = c+ [(key[0],key[1],mfo.regimesDecision[key].solution_value())]
data2b = mfo.data.iloc[mfo.data.index.isin(b)]
data2b.to_csv("solution_alldata_V2_"+RCP+"_"+scenario+"_data.csv")
c1 = pd.DataFrame(c)
c1.to_csv("solution_V2_"+RCP+"_"+scenario+"_solutions.csv")

## Export objective values

The optimal solution for each objective.

In [40]:
with open("objectiveValues_V2_"+scenario+'_'+RCP+".csv","w") as file: 
    delim = "" 
    for objName in mfo.objectiveTypes.keys(): 
        file.write(delim+objName) 
        delim = "," 
    file.write("\n") 
    delim = "" 
    for objName in mfo.objectiveTypes.keys(): 
        file.write(delim+str(mfo.objective[objName].solution_value())) 
        delim = "," 
    file.write("\n")